<a href="https://colab.research.google.com/github/somesh-awasthi/NLP-PROJECT/blob/main/NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
%%capture
!pip install ctransformers==0.2.27 sentence-transformers==2.5.1 chromadb==0.4.24 langchain==0.1.11 PyPDF

#linking my drive with my notebook & giving the data path
from google.colab import drive
drive.mount("/content/drive")
path="/content/drive/MyDrive/Colab Notebooks/data"

import chromadb
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
# from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [34]:
#loading data
def load_data(data):
    loader = DirectoryLoader(data,# glob="*.pdf",
                    glob="symp.pdf",
                    loader_cls=PyPDFLoader)

    documents = loader.load()

    return documents
doc=load_data(path)

In [35]:
#Create text chunks
def text_split(data):
    splitted_text = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    chunk = splitted_text.split_documents(data)

    return chunk
chunk = text_split(doc)
print("length of my chunk:", len(chunk))

length of my chunk: 350


In [36]:
#embedding model
def model():
    embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embedding
embedding = model()
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [46]:
# Creating vector db to store Embeddings
persist_directory = "vectordb"
vectordb = Chroma.from_documents(documents=chunk, embedding=embedding, persist_directory=persist_directory)